# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [1]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [3]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [5]:
spaceship.dropna(inplace=True)
spaceship.isnull().sum()

PassengerId     0
HomePlanet      0
CryoSleep       0
Cabin           0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Name            0
Transported     0
dtype: int64

In [7]:
def reclassify_cabin(cabin):
    if cabin.startswith("A"):
        return "A"
    elif cabin.startswith("B"):
        return "B"
    elif cabin.startswith("C"):
        return "C"
    elif cabin.startswith("D"):
        return "D"
    elif cabin.startswith("E"):
        return "E"
    elif cabin.startswith("F"):
        return "F"
    elif cabin.startswith("G"):
        return "G"
    else:
        return "T"

spaceship["Cabin_Category"] = spaceship["Cabin"].apply(reclassify_cabin) 

In [9]:
spaceship.drop(columns = ["PassengerId", "Name"])

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin_Category
0,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False,B
1,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True,F
2,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,A
3,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,A
4,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,False,A
8689,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,False,G
8690,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,True,G
8691,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,False,E


In [11]:
def reclassify_planet(planet):
    if planet == "Europa":
        return "1"
    elif planet == "Earth":
        return "2"
    else:
        return "3"

spaceship["Planet_Category"] = spaceship["HomePlanet"].apply(reclassify_planet)

In [13]:
def reclassify_cryosleep(cryosleep):
    if cryosleep == True:
        return "1"
    else:
        return "0"

def reclassify_transported(transported):
    if transported == True:
        return "1"
    else:
        return "0"

spaceship["cryosleep _cat"] = spaceship["CryoSleep"].apply(reclassify_cryosleep)
spaceship["transported_cat"] = spaceship["Transported"].apply(reclassify_transported)

In [15]:
features = spaceship.drop(columns = ["PassengerId", "HomePlanet", "CryoSleep", "Cabin", "Destination", "VIP", "Name", "Transported", "Cabin_Category", "transported_cat"])
features.head()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Planet_Category,cryosleep _cat
0,39.0,0.0,0.0,0.0,0.0,0.0,1,0
1,24.0,109.0,9.0,25.0,549.0,44.0,2,0
2,58.0,43.0,3576.0,0.0,6715.0,49.0,1,0
3,33.0,0.0,1283.0,371.0,3329.0,193.0,1,0
4,16.0,303.0,70.0,151.0,565.0,2.0,2,0


In [17]:
target = spaceship[("transported_cat")]
target.head()

0    0
1    1
2    0
3    0
4    1
Name: transported_cat, dtype: object

**Perform Train Test Split**

In [19]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20)

**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

- Bagging and Pasting

In [69]:
bagging_reg = BaggingRegressor(DecisionTreeRegressor(max_depth=20),
                               n_estimators=100,
                               max_samples = 1000)

In [71]:
bagging_reg.fit(X_train, y_train)

BaggingRegressor(estimator=DecisionTreeRegressor(max_depth=20),
                 max_samples=1000, n_estimators=100)

In [73]:
pred = bagging_reg.predict(X_test)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", bagging_reg.score(X_test, y_test))

MAE 0.2761650135094077
RMSE 0.3744352564468289
R2 score 0.43723386553949894


C:\Users\Usuario\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


- Random Forests

In [39]:
forest = RandomForestRegressor(n_estimators=100,
                             max_depth=20)

In [41]:
forest.fit(X_train, y_train)

RandomForestRegressor(max_depth=20)

In [45]:
pred = forest.predict(X_test)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", forest.score(X_test, y_test))

MAE 0.2689391252145746
RMSE 0.38323150779866716
R2 score 0.41048222912048127


C:\Users\Usuario\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


- Gradient Boosting

In [81]:
gb_reg = GradientBoostingRegressor(max_depth=20,
                                   n_estimators=100)

In [83]:
gb_reg.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=20)

In [84]:
pred = gb_reg.predict(X_test)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", gb_reg.score(X_test, y_test))

MAE 0.2796155734529514
RMSE 0.4310941562485779
R2 score 0.2540344746299621


C:\Users\Usuario\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


- Adaptive Boosting

In [47]:
ada_reg = AdaBoostRegressor(DecisionTreeRegressor(max_depth=20),
                            n_estimators=100)

In [49]:
ada_reg.fit(X_train, y_train)

AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=20),
                  n_estimators=100)

In [55]:
pred = ada_reg.predict(X_test)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", ada_reg.score(X_test, y_test))

MAE 0.2670191722861188
RMSE 0.4247840847956352
R2 score 0.2757125513088765


C:\Users\Usuario\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Which model is the best and why?

In [87]:
#KNN was the best model